# Step 1: LUIS App

**Content**

* Create Luis App
* Add Intents
* Add Entities
* Add Utterances
* Train the LUIS model
* Publish the LUIS model
* Test


References:
* https://docs.microsoft.com/en-us/azure/cognitive-services/luis/client-libraries-rest-api?tabs=windows&pivots=programming-language-python
* https://docs.microsoft.com/en-us/azure/cognitive-services/LUIS/luis-concept-entity-types
* https://docs.microsoft.com/en-us/python/api/azure-cognitiveservices-language-luis/azure.cognitiveservices.language.luis.authoring.operations.modeloperations?view=azure-python
* https://github.com/Azure-Samples/cognitive-services-quickstart-code/blob/master/python/LUIS/python-sdk-authoring-prediction/application_quickstart.py

In [21]:
! pip install azure-cognitiveservices-language-luis
! pip install ipykernel
! pip install PyYAML

In [22]:
from azure.cognitiveservices.language.luis.authoring import LUISAuthoringClient
from msrest.authentication import CognitiveServicesCredentials
import math
import datetime, json, os, time
import yaml

 Load Luis Configs

In [23]:
config_file = os.path.join("config","config.yaml")
with open(config_file, 'r') as ymlfile:
    config = yaml.load(ymlfile, Loader=yaml.FullLoader)


In [24]:
authoring_key = config['luis_authoring']['auth_key']
authoring_name = config['luis_authoring']['auth_name']
authoring_endpoint = 'https://{}.cognitiveservices.azure.com/'.format(authoring_name)
authoring_region = config['luis_authoring']['auth_region']
app_name    = config['luis_app']['app_name']
app_desc    = config['luis_app']['app_desc']
app_version = str(config['luis_app']['app_version'])
app_locale  = config['luis_app']['app_locale']

## Create Luis App

In [25]:
# Instantiate a LUIS client
client = LUISAuthoringClient(authoring_endpoint, CognitiveServicesCredentials(authoring_key))


In [ ]:
app_id = None

def create_app(app_name,app_desc, app_version, app_locale):
    # Create a new LUIS app


    app_id = client.apps.add(dict(name=app_name,
                                initial_version_id=app_version,
                                description=app_desc,
                                culture=app_locale))

    print("Created LUIS app {}\n    with ID {}".format(app_name, app_id))
    return app_id, app_version

In [26]:
app_id, app_version = create_app(app_name,app_desc, app_version, app_locale)

Created LUIS app luis-seguradora-app
    with ID ff1edc40-ce40-4de2-bdb8-56d2c519b80f


> **Important**: Fill the *app_id* at *config/config.yml* file:
```yaml
 luis_app:
  app_id: <----
  ```

In [27]:
# Carregando o app id caso precise rodar o notebook a partir desse passo
with open(config_file, 'r') as ymlfile:
    config = yaml.load(ymlfile, Loader=yaml.FullLoader)
app_id = config['luis_app']['app_id']
print(app_id)

ff1edc40-ce40-4de2-bdb8-56d2c519b80f


### Check if the app was created at Luis Portal
* Access [Luis Portal](https://www.luis.ai/)

<img src="media/luis-app.png" alt="Create Azure Account"
	title="Create Azure Account" width="50%" height="50%" />


## Add Intents

Example:
* O número do protocolo é **456748674687**
    * intent: *dizer_protocolo*


In [28]:
def add_intents(app_id, app_version, intents):
    for intent in intents:
        try:
            intentId = client.model.add_intent(app_id, app_version, intent)
            print("Intent {} {} added.".format(intent, intentId))  
            
        except Exception as e:
            if e.__class__.__name__ == 'ErrorResponseException':
                print('Failed to insert intent %s. Reason: Intent already exists!' % (intent))  
            else:
                print(e.message)


In [29]:
intents =  ['classe_roubo','classe_colisao','classe_furto','classe_quebra_retrovisores',
            'dizer_protocolo']

In [30]:
add_intents(app_id, app_version, intents)

Intent classe_roubo 9b80fa04-0408-4de3-9a9a-9f4040255b66 added.
Intent classe_colisao 077f9a99-0a12-404c-bb4a-bd92576278d5 added.
Intent classe_furto 5bf3a332-710b-49e7-9e87-4ca95406e707 added.
Intent classe_quebra_retrovisores 57b8548d-30ed-4259-a6d7-f7e6470193b5 added.
Intent dizer_protocolo 642028a6-3c5d-4031-875e-e9ed156f7d8c added.


### Check if the intents was created at Luis Portal
* Access [Luis Portal](https://www.luis.ai/)
* Click on your Luis app
* Click on **Intents**

<img src="media/luis-intents.png" alt="Create Azure Account"
	title="Create Azure Account" width="50%" height="50%" />



## Add Entities

Example:
* O número do protocolo é **456748674687**
    * intent: *dizer_protocolo*
    * entity:  **456748674687** (*protocolo*)

In [ ]:
def add_entities(app_id, app_version, entity_name, entity_type, regex_pattern=None):
    '''entity_type: 'ml', 'prebuilt','regex' '''
    try:
        if (entity_type == 'ml'):
            client.model.add_entity(app_id, app_version, name=entity_name)
        elif (entity_type == 'prebuilt'):
            client.model.add_prebuilt(app_id, app_version  , prebuilt_extractor_names=[entity_name])
        elif (entity_type == 'regex'):
            client.model.create_regex_entity_model(app_id, app_version, regex_pattern=regex_pattern, name=entity_name)
        else:
            print("Choose one of those entity_types: 'ml', 'prebuilt','regex'")
    except Exception as e:
        if e.__class__.__name__ == 'ErrorResponseException':
            print('Failed to insert entity %s. Reason: Entity already exists!' % (entity_name))  
        else:
            print(e)
    else:
        print ("Entity {} successfully added.".format(entity_name))
        

In [ ]:
#entity_name, entity_type, regex_pattern
entities = [
    ["email", "prebuilt"],
    ["placa", "regex", "[A-Z|a-z]{3}\s[0-9]{4}"],
    ["protocolo", "ml"],
]

In [ ]:
for e in entities:
    reg = ''
    if e[1] == "regex": reg = e[2]
    add_entities(app_id, app_version, e[0],e[1], reg)

### Check if the intents was created at Luis Portal
* Access [Luis Portal](https://www.luis.ai/)
* Click on your Luis app
* Click on **Entities**

<img src="media/luis-entity.png" alt="Create Azure Account"
	title="Create Azure Account" width="50%" height="50%" />


## Add Utterances


Example with entity:
* O número do protocolo é **456748674687**
    * intent: *dizer_protocolo*
    * entity:  **456748674687** (*protocolo*)
    * utterance: O número do protocolo é 456748674687

    ```['dizer_protocolo','o seu protocolo é 456748674687',[('protocolo','456748674687']]```


Example with no entity:
* Roubaram meu carro
    * intent: *classe_roubo*
    * utterance: Roubaram meu carro

    ```['classe_roubo','roubaram meu carro']```


In [ ]:
utterances = [
    ['classe_roubo','roubaram meu carro'],
    ['classe_roubo','meu carro foi roubado'],
    ['classe_roubo','assaltaram meu carro'],
    ['classe_roubo','me roubaram e levaram meu carro'],
    ['classe_roubo','meu carro foi levado no assalto'],
    ['classe_furto','meu carro foi furtado'],
    ['classe_furto','furtaram meu carro'],
    ['classe_furto','meu carro sumiu'],
    ['classe_furto','meu carro desapareceu'],
    ['classe_furto','meu carro não estava mais lá'],
    ['classe_quebra_retrovisores','meu retrovisor quebrou'],
    ['classe_quebra_retrovisores','quebraram o retrovisor do meu carro'],
    ['classe_quebra_retrovisores','meu retrovisor estourou'],
    ['classe_quebra_retrovisores','quebrou o espelho do meu retrovisor'],
    ['classe_quebra_retrovisores','pifou meu retrovisor'],
    ['classe_colisao','bateram no meu carro'],
    ['classe_colisao','bater no meu carro'],
    ['classe_colisao','meu carro foi batido'],
    ['classe_colisao','colidiram em meu carro'],
    ['classe_colisao','houve uma colisão'],
    ['classe_colisao','houve uma pancada em meu carro'],
    ['None','Boa tarde'],
    ['None','como posso te ajudar'],
    ['None','eu preciso de algumas informações para registar o sinistro'],
    ['None',' tudo bem'],
    ['None','Poderia me informar a placa do seu carro'],
    ['None','Qual seria seu  CPF'],
    ['None','qual seria seu telefone com ddd'],
    ['None','E em qual rua aconteceu o roubo'],
    ['None','De qual cidade'],
    ['None','Muito obrigada pelas informações, você poderia anotar o número de protocolo?'],
    ['None','Irei encaminhar seu caso para a área responsável, e eles irão entrar em contato com vocÊ, posso te ajudar em mais alguma coisa'],
    ['None','Boa tarde'],
    ['dizer_protocolo','o número do protocolo é 123456789456 irei agora encaminhar o seu caso para área responsável',[('protocolo','123456789456')]],
    ['dizer_protocolo','o seu protocolo é 123456789456 irei agora encaminhar o seu caso para área responsável',[('protocolo','123456789456')]],
    ['dizer_protocolo','o protocolo é 12345-6789456 irei agora encaminhar o seu caso para área responsável',[('protocolo','12345-6789456')]],
    ['dizer_protocolo','o número de protocolo é 54 04 5795-9221 irei agora encaminhar o seu caso para área responsável',[('protocolo','54 04 5795-9221')]],
    ['dizer_protocolo','o número de protocolo é 5404 5795221 irei agora encaminhar o seu caso para área responsável',[('protocolo','5404 5795221')]],
    ['dizer_protocolo','o número de protocolo é 54 04 5795-9221',[('protocolo','54 04 5795-9221')]]


    
]

In [ ]:
def create_utterance(utterances):
    utterances_luis = list()
    for i in utterances:    
        entities = []
        if len(i) > 2:
            for e in i[2]:
                start = i[1].index(e[1])
                entities.append(dict(entity_name=e[0], 
                                     start_char_index=start,
                                     end_char_index=start + len(e[1])))
        utterances_luis.append({'text': i[1].lower(), 
                                'intent_name' : i[0], 
                                'entity_labels': entities})
    return utterances_luis



In [ ]:
create_utterance(utterances)

In [ ]:
#break the list of utterances because there are a 100 utterances limit batch
qnt_quebras = (math.ceil(len(utterances)/100))
qnt_por_quebras = round(len(utterances)/qnt_quebras)

inicio = 0
proximo = 0
list_quebras = list(range(1,qnt_quebras+1))
broken_utterances = list()
for i in list_quebras:
    if (i == 1):
        proximo = qnt_por_quebras
        ut = utterances[0:proximo]
        broken_utterances.append(ut)
    #a partir do segunda quebra
    else:
        inicio = proximo
        if (i < qnt_quebras):
            proximo = inicio + qnt_por_quebras
        #última quebra
        else:
            proximo = len(utterances)
        ut = utterances[inicio:proximo]
        broken_utterances.append(ut)
    next = qnt_por_quebras + 1

In [ ]:
for u in broken_utterances:
    client.examples.batch(app_id, app_version, create_utterance(u))
    print("{} example utterance(s) added.".format(len(u)))

### Check if the utterances was created at Luis Portal
* Access [Luis Portal](https://www.luis.ai/)
* Click on your Luis app
* Click on **Intents** and check the column **Examples**

<img src="media/luis-utterances.png" alt="Check utterances"
	title="Check utterance" width="50%" height="50%" />

### Check an utterance with entities
* Select some intent with entity

<img src="media/luis-intent-utterance-entity.png" alt="Check an utterance with entities"
	title="Check an utterance with entities" width="50%" height="50%" />

### Check an utterance with no entities
* Select some intent with no entity

<img src="media/luis-intent-utterance.png" alt="Check an utterance with no entities"
	title="Check an utterance with no entities" width="50%" height="50%" />


## Train the LUIS model

In [ ]:
def train_app(app_id, app_version):
    response = client.train.train_version(app_id, app_version)
    waiting = True
    while waiting:
        info = client.train.get_status(app_id, app_version)

        # get_status returns a list of training statuses, one for each model. Loop through them and make sure all are done.
        waiting = any(map(lambda x: 'Queued' == x.details.status or 'InProgress' == x.details.status, info))
        if waiting:
            print ("Waiting 10 seconds for training to complete...")
            time.sleep(10)

In [ ]:
train_app(app_id,app_version)

### Test at Luis Portal \[Alternative]
You can also publish on [Luis Portal](https://www.luis.ai/):
* Click on your Luis app
* Click on **Test**

<img src="media/luis-test.png" alt=""
	title="" width="50%" height="50%" />

## Publish the LUIS model


In [ ]:
def publish_app(app_id, app_version):
    responseEndpointInfo = client.apps.publish(app_id, app_version, is_staging=False,
                                                   speech_priming = True, sentiment_analysis = False)
    print("Application published. Endpoint URL: " + responseEndpointInfo.endpoint_url)
    return responseEndpointInfo.endpoint_url

In [ ]:
endpoint = publish_app(app_id, app_version)

### Publish at Luis Portal \[Alternative]
You can also publish on [Luis Portal](https://www.luis.ai/):
* Click on your Luis app
* Click on **Publish**

<img src="media/luis-publish.png" alt=""
	title="" width="50%" height="50%" />

## Test

In [ ]:
def get_intent(utterance):
    import requests

     # YOUR-APP-ID: The App ID GUID found on the www.luis.ai Application Settings page.
    appId = app_id

    # YOUR-PREDICTION-KEY: Your LUIS authoring key, 32 character value.
    prediction_key = authoring_key

    # YOUR-PREDICTION-ENDPOINT: Replace with your authoring key endpoint.
    # For example, "https://westus.api.cognitive.microsoft.com/"
    prediction_endpoint = "https://{}.api.cognitive.microsoft.com/".format(authoring_region)

    # The utterance you want to use.
    utterance = utterance
    ##########

    try:

        # The headers to use in this REST call.
        headers = {
        }

        # The URL parameters to use in this REST call.
        params ={
            'query': utterance,
            'timezoneOffset': '0',
            'verbose': 'true',
            'show-all-intents': 'true',
            'spellCheck': 'false',
            'staging': 'false',
            'subscription-key': authoring_key
        }

        # Make the REST call.
        response = requests.get(f'{prediction_endpoint}luis/prediction/v3.0/apps/{appId}/slots/production/predict',   headers=headers, params=params)

        # Display the results on the console.
        print(response.json())


    except Exception as e:
        # Display the error string.
        print(f'{e}')

In [ ]:
get_intent("s plsvs GGE 4547")

### Test at Luis Portal \[Alternative]
You can also test on [Luis Portal](https://www.luis.ai/):
* Click on your Luis app
* Click on **Test**

<img src="media/luis-test.png" alt="Check an utterance with entities"
	title="Check an utterance with entities" width="50%" height="50%" />

* Type the utterance to test

<img src="media/luis-testing.png" alt="Check an utterance with entities"
	title="Check an utterance with entities" width="20%" />




## Next Steps
Before testing with Speech Service for the next notebook go to [https://www.luis.ai/] and check the option below:


<img src="media/luis-make-endpoints-public.png" alt="Check an utterance with entities"
	title="Check an utterance with entities" width="50%" />


